In [24]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
#importing the libraries
import numpy as np
import string
from keras.models import Sequential
from keras.layers import LSTM, Dense
import warnings
warnings.filterwarnings("ignore")
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [2]:
# load doc into memory
def load_doc(mylyric):
	# open the file as read only
	file = open(mylyric, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [6]:
# load document
in_filename = 'romance.txt'
doc = load_doc(in_filename)
print(doc[:100])

She's a killer
Looking at ya
She's my type
Very nice to meet ya

She's a killer
One look is all it t


In [7]:
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [9]:
# clean document
tokens = clean_doc(doc)
print(tokens[:10])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['shes', 'a', 'killer', 'looking', 'at', 'ya', 'shes', 'my', 'type', 'very']
Total Tokens: 21995
Unique Tokens: 2186


In [10]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
#print(seq)
#print(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 21944


In [11]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [13]:
 # save sequences to file
out_filename = 'romance_republic_sequences.txt'
save_doc(sequences, out_filename)

In [14]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [17]:
# load
in_filename = 'romance_republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
print(lines[:1])

['shes a killer looking at ya shes my type very nice to meet ya shes a killer one look is all it takes shes a killer and she took my breath away shes a killer romantic lover there is no other she is my lover i need her i gotta see']


In [19]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(sequences[:1])

[[697, 6, 863, 177, 58, 156, 697, 9, 419, 698, 282, 4, 281, 156, 697, 6, 863, 71, 176, 21, 19, 11, 589, 697, 6, 863, 7, 135, 457, 9, 333, 65, 697, 6, 863, 12, 696, 110, 21, 51, 244, 135, 21, 9, 696, 2, 115, 167, 2, 222, 83]]


In [20]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2187


In [22]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
#print(X)
#print(y)
y = to_categorical(y, num_classes=vocab_size)#y,3433
#print(y)
seq_length = X.shape[1]
print(seq_length)

50


In [25]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))#3433,50lengh of vector of each word,#50
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            109350    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 2187)              220887    
                                                                 
Total params: 481,137
Trainable params: 481,137
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=110)

In [27]:
# save the model to file
model.save('romance.h5')
# save the tokenizer
dump(tokenizer, open('romance_tokenizer.pkl', 'wb'))
